### 0. Imports and requirements

* В данном соревновании мы имеем дело с последовательностями, один из интуитивных способов работы с ними - использование рекуррентных сетей. Данный бейзлайн посвящен тому, чтобы показать, как можно строить хорошие решения без использования сложного и трудоемкого feature engineering-а (чтобы эффективно решать ту же задачу с высоким качеством с помощью бустингов нужно несколько тысяч признаков), благодаря рекуррентным сетям. В этом ноутбуке мы построим решение с использованием фреймфорка `torch`. Для комфортной работы Вам понадобится машина с `GPU` (хватит ресурсов `google colab` или `kaggle`).

In [31]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import seaborn as sns
import sys
import pickle
import numpy as np
import torch
import torch.nn as nn
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

pd.set_option('display.max_columns', None)

# добавим корневую папку, в ней лежат все необходимые полезные функции для обработки данных
sys.path.append('../../')
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Data Preprocessing

In [2]:
PATH = '../../data/'

In [ ]:
#preprocecc transactions

transactions = pd.read_csv(PATH + 'transactions.csv', parse_dates=['transaction_dttm'])
clients = pd.read_csv(PATH + 'clients.csv')
report_dates = pd.read_csv(PATH + 'report_dates.csv',parse_dates=['report_dt'])
clients= clients.merge(report_dates, how='left', on='report')

transactions = transactions.merge(clients[['user_id', 'report_dt']], how='left', on='user_id')

transactions['transaction_dttm'] = transactions['transaction_dttm'].astype('datetime64[ns]')
transactions['dweek'] = transactions['transaction_dttm'].dt.dayofweek
transactions['date'] = transactions['transaction_dttm'].dt.date.astype('datetime64[ns]')
transactions['year'] = transactions['transaction_dttm'].dt.year
transactions['year'] = transactions['year'].apply(lambda x: 1 if x == 2022 else 2)
transactions['month'] = transactions['transaction_dttm'].dt.month
transactions['hour'] = transactions['transaction_dttm'].dt.hour
transactions['week'] = transactions['transaction_dttm'].dt.isocalendar()['week']

transactions['days'] = (transactions['report_dt']-transactions['transaction_dttm']).dt.days
transactions['days_dif'] = transactions['days'].diff().fillna(0).abs()

transactions.head()

# Убрем 0 из данных
for col in transactions.columns:
    min = transactions[col].min()
    print(col, 'наимньшее', min)
    if min == 0:
        transactions[col] = transactions[col] + 1 
        min = transactions[col].min()
        print(col, 'наимньшее', min)

transactions.to_csv(PATH+'transactions_mod.csv', index=None)

In [3]:
features = ['user_id', 'mcc_code', 'currency_rk', 'transaction_amt',
       'transaction_dttm', 'report_dt', 'dweek', 'date', 'year', 'month',
       'hour', 'week', 'days', 'days_dif']

* Как и в случае с бустингами, мы не можем поместить всю выборку в память, в виду, например, ограниченных ресурсов. Для итеративного чтения данных нам потребуется функция `utils.read_parquet_dataset_from_local`, которая читает N частей датасета за раз в память.


* Нейронные сети требуют отделнього внимания к тому, как будут поданы и обработаны данные. Важные моменты, на которые требуется обратить внимание:
    * Использование рекуррентных сетей подразумевает работу на уровне последовательностей, где одна последовательность - все исторические транзакции клиента. Чтобы преобразовать `pd.DataFrame` с транзакциями клиентов в табличном виде к последовательностям, мы подготовили функцию `dataset_preprocessing_utils.transform_transactions_to_sequences`, она делает необходимые манипуляции и возвращает фрейм с двумя колонками: `app_id` и `sequences`. Колонка `sequence` представляет из себя массив массивов длины `len(features)`, где каждый вложенный массив - значения одного конкретного признака во всех транзакциях клиента. 
    
    * каждый клиент имеет различную по длине историю транзакций. При этом обучение сетей происходит батчами, что требует делать паддинги в последовательностях. Довольно неэффективно делать паддинг внутри батча на последовательностях случайной длины (довольно часто будем делать большой и бесполезный паддинг). Гораздо лучше использовать технику `sequence_bucketing` (о ней рассказано в образовательном ролике к данному бейзлайну). Для этого мы предоставляем функцию `dataset_preprocessing_utils.create_padded_buckets`. Один из аргументов в данную функцию - `bucket_info` - словарь, где для конкретной длины последовательности указано до какой длины нужно делать паддинг. Мы предоставялем для старта простой вид разбиения на 100 бакетов и файл где лежит отображение каждой длины в падднг (файл `buckets_info.pkl`).
    
    * Такие признаки, как [`amnt`, `days_before`, `hour_diff`] по своей природе не являются категориальными. Вы в праве самостоятельно выбирать способ работы с ними (модифицируя функции бейзлайна или адаптируя под себя). В рамках бейзлайна мы предлагаем интерпретировать каждую не категориальную фичу как категориальную. Для этого нужно подготовить bin-ы для каждой фичи. Мы предлагаем простой способ разбиения по бинам.

In [3]:
# from utils import read_parquet_dataset_from_local
from dataset_preprocessing_utils import transform_transactions_to_sequences, create_padded_buckets

In [4]:
import pickle

with open('../buckets_info.pkl', 'rb') as f:
    mapping_seq_len_to_padded_len = pickle.load(f)
    
with open('../dense_features_buckets.pkl', 'rb') as f:
    dense_features_buckets = pickle.load(f)

In [65]:
# mapping_seq_len_to_padded_len

In [ ]:
# # make bucket info

# mapping_seq = seq['sequence_length'].quantile([0.25,0.5,0.75]).values

# mapping_seq= mapping_seq.astype(int)

# mapping = {}
# for sequence in seq['sequence_length'].values:
#     if sequence <= mapping_seq[0]:
#         mapping[sequence] = mapping_seq[0]
#     elif sequence <= mapping_seq[1]:
#         mapping[sequence] = mapping_seq[1]
#     else:
#         mapping[sequence] = mapping_seq[2]
    
# mapping_seq_len_to_padded_len = mapping

# with open('../buckets_info.pkl', 'wb') as f:
#     pickle.dump(mapping_seq_len_to_padded_len, f)

In [ ]:
# # make dense_features_buckets

# dense_features_buckets = {'transaction_amt':[-2407.71005859,  -981.33168597,  -558.5010376 ,  -359.30341666,
#         -230.8977805 ,  -141.41950509,   -65.03555559,   0]}

# with open('../dense_features_buckets.pkl', 'wb') as f:
#     pickle.dump(dense_features_buckets, f)

In [ ]:
# transactions['transaction_amt'].quantile(np.linspace(0.1,0.95, 8)).values

In [ ]:
# train = pd.read_csv(PATH + '/train.csv')
# seq = seq.merge(train, how='left', on='user_id')
# seq

In [5]:
from sklearn.model_selection import StratifiedKFold
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train = pd.read_csv(PATH + '/train.csv')

for split, (train_index, valid_index) in enumerate(strat_kfold.split(train, train['target'])):
    X_train, X_val = train.iloc[train_index], train.iloc[valid_index]
    print(split)
    print(train_index)
    print(valid_index)
    print('*'*25)


0
[    1     2     3 ... 63996 63998 63999]
[    0    11    29 ... 63990 63993 63997]
*************************
1
[    0     1     3 ... 63996 63997 63999]
[    2     4     5 ... 63985 63989 63998]
*************************
2
[    0     1     2 ... 63997 63998 63999]
[    7     8    16 ... 63963 63966 63991]
*************************
3
[    0     2     3 ... 63997 63998 63999]
[    1     6    10 ... 63988 63994 63996]
*************************
4
[    0     1     2 ... 63996 63997 63998]
[    3    12    15 ... 63992 63995 63999]
*************************


In [97]:
X_val

,user_id,target,time
3,41,0,57
12,76,0,78
15,82,0,91
17,94,0,91
21,140,0,91
...,...,...,...
63978,559523,0,74
63984,560300,0,91
63992,561033,0,91
63995,561824,0,91


In [51]:
X_val

,user_id,target,time
3,41,0,57
12,76,0,78
15,82,0,91
17,94,0,91
21,140,0,91
...,...,...,...
63978,559523,0,74
63984,560300,0,91
63992,561033,0,91
63995,561824,0,91


In [7]:
! mkdir ./processed_chunk/

In [16]:
def create_buckets_from_transactions(path = PATH, save_to_path = './processed_chunk/'):

    transactions_frame = pd.read_csv(path + 'transactions_mod.csv', parse_dates=['transaction_dttm'])
    for dense_col in ['transaction_amt']:
        transactions_frame[dense_col] = np.digitize(transactions_frame[dense_col], bins=dense_features_buckets[dense_col])
        
    seq = transform_transactions_to_sequences(transactions_frame)
    seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[1]))

    train = pd.read_csv(path + '/train.csv')
    seq = seq.merge(train, how='left', on='user_id')
    seq_train = seq[seq['target'].notna()]
    seq_test = seq[seq['target'].isna()]

    # for split in range(5):
    #     random_state = 10*split
    #     X_train, X_val = train_test_split(seq_train, random_state=random_state, test_size=0.25)
    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for split, (train_index, valid_index) in enumerate(strat_kfold.split(seq_train, seq_train['target'])):
        X_train, X_val = seq_train.iloc[train_index], seq_train.iloc[valid_index]

        processed_train =  create_padded_buckets(X_train, mapping_seq_len_to_padded_len, has_target=True, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                f'processed_chunk_train_{split}.pkl'))
        processed_valid = create_padded_buckets(X_val, mapping_seq_len_to_padded_len, has_target=True, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                f'processed_chunk_valid_{split}.pkl'))
        
    processed_test =  create_padded_buckets(seq_test, mapping_seq_len_to_padded_len, has_target=False, 
                                                save_to_file_path=os.path.join(save_to_path, 
                                                                            f'processed_chunk_test.pkl'))
    

    

In [ ]:
    # strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # for split, (train_index, valid_index) in enumerate(strat_kfold.split(seq_train, seq_train['target'])):
    #     X_train, X_val = seq_train.iloc[train_index], seq_train.iloc[valid_index]

In [17]:
create_buckets_from_transactions()

/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences['bucket_idx'] = 186 #frame_of_sequences.sequence_length.map(bucket_info)


Extracting buckets:   0%|          | 0/1 [00:00<?, ?it/s]

/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)
/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences['bucket_idx'] = 186 #frame_of_sequences.sequence_length.map(bucket_info)


Extracting buckets:   0%|          | 0/1 [00:00<?, ?it/s]

/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)
/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences['bucket_idx'] = 186 #frame_of_sequences.sequence_length.map(bucket_info)


...

/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)
/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences['bucket_idx'] = 186 #frame_of_sequences.sequence_length.map(bucket_info)


Extracting buckets:   0%|          | 0/1 [00:00<?, ?it/s]

/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)
/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences['bucket_idx'] = 186 #frame_of_sequences.sequence_length.map(bucket_info)


Extracting buckets:   0%|          | 0/1 [00:00<?, ?it/s]

/media/andrey/HDD/Study/ods_data_fusion_2024/rnn_andvanced/baseline/../dataset_preprocessing_utils.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)


* Разобьем имеющиеся данные на `train` и `val` части. Воспользуемся самым простым способом - для валидации используем 10% случайных данных

In [18]:
import gc
gc.collect()

595

### 2. Modeling

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cuda


* Для создания модели будем использовать фреймворк `torch`. В нем есть все, чтобы писать произвольные сложные архитектуры и быстро эксперементировать. Для того, чтобы мониторить и логировать весь процесс во время обучения сетей, рекомендуется использовать надстройки над данным фреймворков, например, `lightning`.

* В бейзлайне мы предлагаем базовые компоненты, чтобы можно было обучать нейронную сеть и отслеживать ее качество. Для этого вам предоставлены следующие функции:
    * `data_generators.batches_generator` - функция-генератор, итеративно возвращает батчи, поддерживает батчи для `tensorflow.keras` и `torch.nn.module` моделей. В зависимости от флага `is_train` может быть использована для генерации батчей на train/val/test стадию.
    * функция `pytorch_training.train_epoch` - обучает модель одну эпоху.
    * функция `pytorch_training.eval_model` - проверяет качество модели на отложенной выборке и возвращает roc_auc_score.
    * функция `pytorch_training.inference` - делает предикты на новых данных и готовит фрейм для проверяющей системы.
    * класс `training_aux.EarlyStopping` - реализует early_stopping, сохраняя лучшую модель. Пример использования приведен ниже.

In [20]:
from data_generators import batches_generator, transaction_features
from pytorch_training import train_epoch, eval_model, inference
from training_aux import EarlyStopping

* Все признаки в нашей модели будут категориальными. Для их представления в модели используем категориальные эмбеддинги. Для этого нужно каждому категориальному признаку задать размерность латентного пространства. Используем [формулу](https://forums.fast.ai/t/size-of-embedding-for-categorical-variables/42608) из библиотеки `fast.ai`. Все отображения хранятся в файле `embedding_projections.pkl`

In [ ]:
# with open('../embedding_projections.pkl', 'rb') as f:
#     embedding_projections = pickle.load(f)

# embedding_projections

In [40]:
for col in transactions.columns:
    print(col, 'unique', transactions[col].max())


user_id unique 562740
mcc_code unique 450
currency_rk unique 4
transaction_amt unique 352076.8125
transaction_dttm unique 2023-03-20 20:59:58
report_dt unique 2023-06-30 03:00:00
dweek unique 7
date unique 2023-03-20
year unique 2023
month unique 12
hour unique 24
week unique 52
days unique 283
days_dif unique 183.0


In [9]:
transaction_features

['mcc_code',
 'currency_rk',
 'transaction_amt',
 'dweek',
 'year',
 'month',
 'hour',
 'week',
 'days',
 'days_dif']

In [15]:
So it looks like it changed since I last looked. Previously, it was:

def emb_sz_rule(n_cat:int)->int: return min(50, (n_cat//2)+1)

Now it looks like it’s the following:

def emb_sz_rule(n_cat:int)->int: return min(600, round(1.6 * n_cat**0.56))


226

In [21]:
embedding_projections = {'mcc_code': (450, 200), 'currency_rk': (4,3), 'transaction_amt': (10,6),
                         'dweek': (7,4), 'year':(2,2), 'month':(12,7), 'hour':(24,13), 'week': (52,27),  'days': (283,141), 'days_dif':(183,92)}


* Реализуем модель. Все входные признаки представим в виде эмбеддингов, сконкатенируем, чтобы получить векторное представление транзакции. Подадим последовательности в `GRU` рекуррентную сеть. Используем последнее скрытое состояние в качестве выхода сети. Представим признак `product` в виде отдельного эмбеддинга. Сконкатенируем его с выходом сети. На основе такого входа построим небольшой `MLP`, выступающий классификатором для целевой задачи. Используем градиентный спуск, чтобы решить оптимизационную задачу. 

In [22]:
class TransactionsRnn(nn.Module):
    def __init__(self, transactions_cat_features, embedding_projections, product_col_name='product', rnn_units=128, top_classifier_units=32):
        super(TransactionsRnn, self).__init__()
        self._transaction_cat_embeddings = nn.ModuleList([self._create_embedding_projection(*embedding_projections[feature]) 
                                                          for feature in transactions_cat_features])
                
        # self._product_embedding = self._create_embedding_projection(*embedding_projections[product_col_name], padding_idx=None)
        
        self._gru = nn.GRU(input_size=sum([embedding_projections[x][1] for x in transactions_cat_features]),
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        self._hidden_size = rnn_units
                
        self._top_classifier = nn.Linear(in_features=rnn_units, 
                                         out_features=top_classifier_units)
        self._intermediate_activation = nn.ReLU()
        
        self._head = nn.Linear(in_features=top_classifier_units, out_features=1)
    
    def forward(self, transactions_cat_features):
        batch_size = 32
        
        embeddings = [embedding(transactions_cat_features[i]) for i, embedding in enumerate(self._transaction_cat_embeddings)]
        concated_embeddings = torch.cat(embeddings, dim=-1)
        
        _, last_hidden = self._gru(concated_embeddings)
        last_hidden = torch.reshape(last_hidden.permute(1, 2, 0), shape=(batch_size, self._hidden_size))
        
        # product_embed = self._product_embedding(product_feature)
        
        intermediate_concat = torch.cat([last_hidden], dim=-1)
                
        classification_hidden = self._top_classifier(intermediate_concat)
        activation = self._intermediate_activation(classification_hidden)
        
        logit = self._head(activation)
        
        return logit
    
    @classmethod
    def _create_embedding_projection(cls, cardinality, embed_size, add_missing=True, padding_idx=0):
        add_missing = 1 if add_missing else 0
        return nn.Embedding(num_embeddings=cardinality+add_missing, embedding_dim=embed_size, padding_idx=padding_idx)


### 3. Training

In [111]:
! mkdir ./checkpoints/

! rm -r ./checkpoints/pytorch_baseline/
! mkdir ./checkpoints/pytorch_baseline

mkdir: cannot create directory ‘./checkpoints/’: File exists


In [23]:
! mkdir ./checkpoints/kfold

* Для того, чтобы детектировать переобучение используем EarlyStopping.

In [24]:
path_to_checkpoints = './checkpoints/kfold/'
es = EarlyStopping(patience=3, mode='max', verbose=True, save_path=os.path.join(path_to_checkpoints, 'best_checkpoint.pt'), 
                   metric_name='ROC-AUC', save_format='torch')

In [25]:
num_epochs = 3
train_batch_size = 32
val_batch_szie = 32

In [ ]:


for i in range(5):
    with open(f'./processed_chunk/processed_chunk_train_{i}.pkl', 'rb') as f:
        dataset_train = pickle.load(f)  
    with open(f'./processed_chunk/processed_chunk_valid_{i}.pkl', 'rb') as f:
        dataset_valid = pickle.load(f)     
        
    model = TransactionsRnn(transaction_features, embedding_projections).to(device)
    optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
    
    for epoch in range(num_epochs):
        print(f'Starting epoch {epoch+1}')
        train_epoch(model, optimizer, dataset_train, batch_size=train_batch_size, 
                    print_loss_every_n_batches=100, device=device)
        
        val_roc_auc = eval_model(model, dataset_valid, batch_size=val_batch_szie, device=device)
        # es(val_roc_auc, model)
        
        # if es.early_stop:
        #     print('Early stopping reached. Stop training...')
        #     break
        torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}_split_{i}.pt'))
        
        train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_szie, device=device)
        print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')


* Запустим цикл обучения, каждую эпоху будем логировать лосс, а так же roc-auc на валидации и на обучении. Будем сохрнаять веса после каждой эпохи, а так же лучшие с помощью early_stopping.

### 4. Submission

* Все готово, чтобы сделать предсказания для тестовой выборки. Нужно только подготовить данные в том же формате, как и для train.

In [ ]:
! rm -r ../../../test_buckets
! mkdir ../../../test_buckets

* Отдельный вопрос, какую из построенных моделей использовать для того, чтобы делать предсказания на тест. Можно выбирать лучшую по early_stopping. В таком случае есть риск, что мы подгонимся под валидационную выборку, особенно если она не является очень репрезентативной, однако это самый базовый вариант (используем его). Можно делать разные версии ансамблирования, используя веса с разных эпох. Такой подход требует дополнительного кода (обязательно попробуйте его!). Наконец, можно выбирать такую модель, которая показывает хорошие результаты на валидации и в то же время, не слишком переучена под train выборку.

In [28]:
! ls $path_to_checkpoints

epoch_1_val_0.649_split_3.pt  epoch_2_val_0.672_split_1.pt
epoch_1_val_0.665_split_1.pt  epoch_2_val_0.681_split_2.pt
epoch_1_val_0.671_split_0.pt  epoch_3_val_0.655_split_3.pt
epoch_1_val_0.673_split_4.pt  epoch_3_val_0.657_split_1.pt
epoch_1_val_0.682_split_2.pt  epoch_3_val_0.660_split_0.pt
epoch_2_val_0.654_split_3.pt  epoch_3_val_0.667_split_4.pt
epoch_2_val_0.665_split_0.pt  epoch_3_val_0.673_split_2.pt
epoch_2_val_0.669_split_4.pt


In [29]:
import glob
glob.glob(path_to_checkpoints + 'epoch_2*')

['./checkpoints/kfold/epoch_2_val_0.654_split_3.pt',
 './checkpoints/kfold/epoch_2_val_0.681_split_2.pt',
 './checkpoints/kfold/epoch_2_val_0.672_split_1.pt',
 './checkpoints/kfold/epoch_2_val_0.665_split_0.pt',
 './checkpoints/kfold/epoch_2_val_0.669_split_4.pt']

In [ ]:
score = []
for i in range(5):
    with open(f'./processed_chunk_valid_{i}.pkl', 'rb') as f:
        dataset_valid = pickle.load(f) 

    sample = pd.read_csv(PATH + 'sample_submit_naive.csv')
    sample['predict'] = 0
    out = np.zeros(16000)
    for i in glob.glob(path_to_checkpoints + 'epoch_2*'):
        model.load_state_dict(torch.load(i))
        test_preds = inference(model, dataset_valid, batch_size=32, device=device)
        
        out += test_preds['predict'].values
        score.append(metrics.roc_auc_score(dataset_valid['targets'][0], out/5))


np.mean(score)

In [31]:
test_preds.head()

,user_id,predict
0,9,0.031792
1,61,0.025088
2,62,0.003125
3,80,0.030441
4,88,0.088427


In [37]:

with open(f'./processed_chunk_test.pkl', 'rb') as f:
    dataset_test = pickle.load(f) 

sample = pd.read_csv(PATH + 'sample_submit_naive.csv')
sample['predict'] = 0
for i in glob.glob(path_to_checkpoints + 'epoch_2*'):
    model.load_state_dict(torch.load(i))
    test_preds = inference(model, dataset_test, batch_size=32, device=device)
    
    sample['predict'] =  sample['predict'] +test_preds['predict'].values

sample['predict'] = sample['predict']


sample

Test time predictions: 0it [00:00, ?it/s]

Test time predictions: 0it [00:00, ?it/s]

Test time predictions: 0it [00:00, ?it/s]

Test time predictions: 0it [00:00, ?it/s]

Test time predictions: 0it [00:00, ?it/s]

,user_id,predict
0,9,0.298615
1,61,0.777377
2,62,0.236845
3,80,0.655628
4,88,0.928605
...,...,...
31995,561362,0.247617
31996,561419,0.394992
31997,561895,0.589917
31998,561908,0.768405


In [40]:
sample['predict'] = sample['predict']
sample

,user_id,predict
0,9,0.059723
1,61,0.155475
2,62,0.047369
3,80,0.131126
4,88,0.185721
...,...,...
31995,561362,0.049523
31996,561419,0.078998
31997,561895,0.117983
31998,561908,0.153681


In [44]:
sample.to_csv('rnn_baseline_submission.csv', index=None) # ~ 0.750 на public test

In [47]:
submit = pd.read_csv('../../baseline/submit_baseline_0.770.csv')
submit
# submit['predict'] = 0.7*submit['predict']+0.2*sample['predict'] # 0.775
submit['predict'] = 0.9*submit['predict']+0.1*sample['predict'] # 0.775
# submit['predict'] = 0.5*submit['predict']+0.5*test_preds['predict'] # 0.771

submit.to_csv('rnn_baseline_stacked_09_01.csv', index=None)

In [48]:
submit

,user_id,predict
0,9,0.082403
1,61,0.113720
2,62,0.243039
3,80,0.037259
4,88,0.611515
...,...,...
31995,561362,0.278235
31996,561419,0.279356
31997,561895,0.217220
31998,561908,0.329889
